In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, glob
import matplotlib
import numpy as np
from geopandas.tools import geocode
import time
import os, json, urllib, requests, webbrowser


In [2]:
df_desc = pd.read_excel('./data/datasets_description.xlsx')
df_desc.head()

,Database,Description,Output
0,Aquifer_Auser,Information about the Auser aquifer. This wate...,"Depth_to_Groundwater_SAL, Depth_to_Groundwater..."
1,Water_Spring_Amiata,Information about the Amiata aquifer. This aqu...,"Flow_Rate_Bugnano, Flow_Rate_Arbure,\n Flow_Ra..."
2,Aquifer_Petrignano,Information about Petrignano aquifer. \nIt is ...,"Depth_to_Groundwater_P24,\n Depth_to_Groundwat..."
3,Aquifer_Doganella,Information about Doganella aquifer. The Dogan...,"Depth_to_Groundwater_Pozzo_1, Depth_to_Groundw..."
4,Aquifer_Luco,Information about Luco aquifer. It is an under...,Depth_to_Groundwater_Podere_Casetta


In [3]:
## collect all geolocations of header names

# todo: manually check and fix, some don't seem correct (Laghetto Verde should prob be: 44.827716,7.0187831)
if os.path.exists('./data/geolocations.pckl'):
    gdfs = pd.read_pickle('./data/geolocations.pckl')
else:

    cols = []
    for file in glob.glob('./data/*.csv'):
        df = pd.read_csv(file)
        [cols.append(c) for c in df.columns]
    cols = [c for c in set(cols) if 'Depth' not in c and 'Flow_Rate' not in c and 'Volume' not in c and 'Lake_Level' not in c]
    cols = set(cols) - set(['Date'])
    cols = [c.replace('Rainfall', '').replace('Temperature', '').replace('Hydrometry', '').replace('_', ' ').strip() for c in cols]
    places = cols
    places = set(places)

    locs = ["{}, Italy".format(name) for name in places]
    gdfs = []

    gdfs = geocode(list(locs))

    gdfs['place'] = places
    gdfs.to_pickle('./data/geolocations.pckl')
    gdfs.head()


In [5]:
# collect some relevant metrics from nasa POWER

output = "JSON"
locations = [gdfs.geometry.values]

output_folder = r'./data/nasa-power'
params = 'QV2M,RH2M,PS,PRECTOT,T2M_RANGE,T2MDEW,T2MWET,WS10M_RANGE,WS50M_RANGE'
base_url = "https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=SinglePoint&tempAverage=DAILY&parameters={}&startDate=20000101&endDate=20210101&lat={latitude}&lon={longitude}&outputList={output}&userCommunity=SSE"
gdfs['longitude'] = gdfs.geometry.x
gdfs['latitude'] = gdfs.geometry.y

for i, row in gdfs.iloc[:].iterrows():
    longitude = row.longitude
    latitude = row.latitude
    api_request_url = base_url.format(params, longitude=longitude, latitude=latitude, output=output.upper())

    json_response = json.loads(requests.get(api_request_url).content.decode('utf-8'))

    # Selects the file URL from the JSON response
    csv_request_url = json_response['outputs'][output.lower()]

    # Download File to Folder
    output_file_location = os.path.join(output_folder, row.place)+'.json'
    urllib.request.urlretrieve(csv_request_url, output_file_location)

In [32]:
for f in glob.glob('./data/nasa-power/*.json'):
    with open(f) as file:
        data = json.load(file)
    df = pd.DataFrame(data['features'][0]['properties']['parameter'])
    df.index = pd.to_datetime(df.index)
    df.to_pickle(f.replace('.json', '.pckl'))